In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt

# Entrenamiento

In [4]:
data = pd.read_csv('../age_gender.csv')

In [5]:
data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32').reshape(48, 48))# Cambia a la ruta de tu archivo CSV

In [6]:
X = np.stack(data['pixels'].values) / 255.0
X = X.reshape(-1, 48, 48, 1)

In [7]:
y = data['age'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Cambiar a 1 salida y activación lineal
])

NameError: name 'Conv2D' is not defined

In [35]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10


C:\My programers\Anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


593/593 ━━━━━━━━━━━━━━━━━━━━ 25s 40ms/step - loss: 482.3643 - mae: 16.8264 - val_loss: 258.5425 - val_mae: 11.9199
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - loss: 262.8578 - mae: 12.4478 - val_loss: 380.3854 - val_mae: 14.8158
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 243.6750 - mae: 11.8848 - val_loss: 189.0170 - val_mae: 10.1210
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - loss: 217.3691 - mae: 11.1208 - val_loss: 202.2165 - val_mae: 10.4049
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 24s 41ms/step - loss: 195.2475 - mae: 10.5183 - val_loss: 181.9973 - val_mae: 9.7978
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 24s 41ms/step - loss: 193.6530 - mae: 10.4838 - val_loss: 159.9074 - val_mae: 9.1654
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - loss: 177.3729 - mae: 9.9689 - val_loss: 134.1991 - val_mae: 8.4859
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - loss: 170.9685 - mae: 9.7934 - val_loss: 167.1152 - val_mae: 9.3875
Epoch 9/1

In [ ]:
model.save('modelo_prediccion_edad.h5')

# Evaluacion

In [11]:
model = load_model("../modelos/age_model.h5", custom_objects={'mse': MeanSquaredError()})

In [14]:
resultados = model.evaluate(X_test, y_test, verbose=2)

149/149 - 1s - 7ms/step - loss: 100.9625 - mean_absolute_error: 7.3184


In [44]:
print("Pérdida (Loss):", resultados[0])
print("MAE:", resultados[1])

149/149 - 2s - 13ms/step - loss: 78.9112 - mean_absolute_error: 6.5996
Pérdida (Loss): 78.91122436523438
MAE: 6.599600791931152


In [19]:
def predecir(imagen):
    pixels = imagen['pixels'].values[0]
    if isinstance(pixels, str):
        pixels = np.array([int(i) for i in pixels.split()])
    elif isinstance(pixels, np.ndarray):
        pixels = pixels.astype(int)
    pixels = pixels.reshape(48, 48)
    pixels = np.expand_dims(pixels, axis=(0, -1)) / 255.0
    return model.predict(pixels)[0][0]

In [23]:
random_row = data.sample(1)
real_age = random_row['age'].values[0]
predicted_age = predecir(random_row)
print(f'Real Age: {real_age}, Predicted Age: {predicted_age:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Real Age: 47, Predicted Age: 37.89


In [24]:
random_row = data.sample(1)
real_age = random_row['age'].values[0]
predicted_age = predecir(random_row)
print(f'Real Age: {real_age}, Predicted Age: {predicted_age:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Real Age: 18, Predicted Age: 21.34


In [25]:
random_row = data.sample(1)
real_age = random_row['age'].values[0]
predicted_age = predecir(random_row)
print(f'Real Age: {real_age}, Predicted Age: {predicted_age:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Real Age: 58, Predicted Age: 54.12
